# 🐡 The sharp points of Rockpool 🐚

## 🦀 How to use sampled time series data (in)correctly in Rockpool

## 🐚 Defining extents for `TSEvent` time series data

Make a figure showing events occurring in real time, versus an event raster.